In [1]:
# Useful starting lines
%matplotlib inline

import numpy as np
import scipy
import scipy.io
import scipy.sparse as sp
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [ ]:
from helpers import *
from cost import *
crossValidation = True
if crossValidation:
    trainDF, testDF = crossValidationSets(0.7)
    testDF.sort_values(['User', 'Movie'], ascending=[1,1], inplace=True)
else:
    trainDF, testDF = realDataSets() # Kaggle datasets (with no predictions in valuesTest)

print(trainDF.head())
print(testDF.head())

number of items: 10000, number of users: 1000
         User  Movie  Prediction
768260    625   7331         5.0
799019    644   6906         4.0
337150    284   5362         3.0
822526    657   9880         4.0
1102454   907   8468         3.0
    User  Movie  Prediction
1      0     60         3.0
5      0     89         4.0
12     0    181         3.0
16     0    354         2.0
18     0    400         4.0


In [ ]:
from predictionAlgorithms import *

rankALS = 6
numIterationsALS = 20
lambdaALS = 0.07585775750291851

rankALS = 8
lambdaALS = 0.081

print("User mean")
predictionsUserMean = baseline_user_mean(trainDF, testDF)
print("Item mean")
predictionsItemMean = baseline_item_mean(trainDF, testDF)
print("ALS Spark")
#predictionsALS = ALSPyspark(trainDF, testDF, rankALS, lambdaALS, numIterationsALS)
predictionsALS = ALSPyspark(trainDF, testDF, rankALS, lambdaALS, numIterationsALS)
print("ALS with bias pywFM")
predictionsALSBias = ALSBias_pywFM(trainDF, testDF)
print("MCMC")
predictionsMCMC = MCMC_pywFM(trainDF, testDF)

User mean
Item mean
ALS Spark


In [ ]:
print("ALS with bias pywFM")
predictionsALSBias = ALSBias_pywFM(trainDF, testDF)
print("MCMC")
predictionsMCMC = MCMC_pywFM(trainDF, testDF)

In [ ]:
#print(predictionsALS.Prediction.values)
print(predictionsALS.head())
print(testDF.head())

In [ ]:
print(predictionsALS.head())
testDF.sort_values(['User', 'Movie'], ascending=[1,1], inplace=True)
print(testDF.head())
#print(predictionsALS.Prediction)
#print(testDF.Prediction)
print(compute_cost(predictionsALS._3, trainDF.Prediction.values))
if crossValidation:
    #print(np.mean((predictionsALS._3-testDF.Prediction.values)**2))
    print(compute_cost(predictionsALS._3, testDF.Prediction.values))
    #print(np.mean((predictionsALSMartin._3.values-testDF.Prediction)**2))

In [ ]:
0.1*np.array(predictionsALSBias)
#0.1 *predictionsUserMean

In [ ]:
testDF

In [ ]:
testDF.sort_values(['User', 'Movie'], ascending=[1,1], inplace=True)
testDF

In [ ]:
#wUserMean = 0.0204081632653
#wItemMean = 0.020842379505
#wALS = 1-wUserMean-wItemMean

wUserMean = 0.05
wItemMean = 0.
wALS = 0.2
wALSBias = 0.3
wMCMC = 0.6

#predictions = (wUserMean*predictionsUserMean + wItemMean*predictionsItemMean + wALS*predictionsALS['Prediction']) / (wUserMean + wItemMean + wALS)
predictions = (wUserMean*predictionsUserMean + wItemMean*predictionsItemMean + wALS*predictionsALS._3.values +
               wALSBias * np.array(predictionsALSBias) + wMCMC * np.array(predictionsMCMC)) / (wUserMean + wItemMean + wALS + wALSBias + wMCMC)
print(predictionsUserMean[0])
print(predictionsItemMean[0])
#print(predictionsALS.Prediction.values[0])
print(predictionsALS._3.values[0])
print(predictionsALSBias[0])
print(predictionsMCMC[0])
print(predictions[0])

#compute_cost(predictions, testDF['Prediction'])

In [ ]:
from cv import *
wUser, wItem, wALS, wALSBias, wMCMC, loss = optimize_weights(predictionsUserMean, predictionsItemMean, predictionsALS._3.values, np.array(predictionsALSBias), np.array(predictionsMCMC), testDF.Prediction.values)
print(wUser)
print(wItem)
print(wALS)
print(wALSBias)
print(wMCMC)
print(loss)

In [ ]:
from helpers import *
make_submission(predictions)

In [ ]:
predictionsALS

In [ ]:
from predictionAlgorithms import *
predictionsALS = ALSPyspark(trainDF, testDF)

In [ ]:
predictionsALS.columns = ['User', 'Movie', 'Prediction']
predictionsALS.sort_values(['User', 'Movie'], ascending=[1,1], inplace=True)
predictionsALS.head()

In [ ]:
predictions = 0.25*predictionsUserMean+0.25*predictionsItemMean + 0.5*predictionsALS['Prediction']

In [ ]:
make_submission(predictions)